In [4]:
import numpy as np 
import matplotlib.pyplot as plt
import glob
import cv2
import shutil
from keras.models import Model, Sequential

ModuleNotFoundError: No module named 'tensorflow'

In [5]:

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (
    BatchNormalization, SeparableConv2D, MaxPooling2D, Activation, Flatten, Dropout, Dense
)
import os
import seaborn as sns
from keras.applications.vgg16 import VGG16
from zipfile import ZipFile

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
with ZipFile("/content/drive/MyDrive/dogs_images/archive.zip", 'r') as zip_ref:
  zip_ref.extractall("/archive")

In [7]:
breed_list = os.listdir("../archive/images/Images/")
standard_path = "../archive/images/Images/"
validation_path = "../archive/images/validations/"
train_path = "../archive/images/train/"

In [ ]:
# Path de validations
paths = []
labels = []
targets = []
for breed in breed_list:
    base_name = os.path.join(standard_path, breed)
    os.mkdir(os.path.join(validation_path, breed))



In [ ]:
# Path de train
paths = []
labels = []
targets = []
for breed in breed_list:
    base_name = os.path.join(standard_path, breed)
    os.mkdir(os.path.join(train_path, breed))


In [ ]:
# Duplicata des images de validations
for breed in breed_list:
    base_name = os.path.join(standard_path, breed)
    count = 0
    for img_name in os.listdir(base_name):
      if count > 29:
        break
      shutil.copy2(os.path.join(base_name, img_name), os.path.join(validation_path, breed))
      count+=1

In [ ]:
# Duplicata des images de trains
for breed in breed_list:
    base_name = os.path.join(standard_path, breed)
    count = 0
    for img_name in os.listdir(base_name):
      if count > 99:
        break
      shutil.copy2(os.path.join(base_name, img_name), os.path.join(train_path, breed))
      count+=1

In [8]:
#Capture training data and labels into respective lists
train_images = []
train_labels = [] 
SIZE = 256

In [46]:
print(glob.glob(train_path+"*"))

['../archive/images/train/n02106382-Bouvier_des_Flandres', '../archive/images/train/n02102177-Welsh_springer_spaniel', '../archive/images/train/n02085620-Chihuahua', '../archive/images/train/n02110627-affenpinscher', '../archive/images/train/n02088364-beagle', '../archive/images/train/n02092002-Scottish_deerhound', '../archive/images/train/n02097658-silky_terrier', '../archive/images/train/n02107683-Bernese_mountain_dog', '../archive/images/train/n02095570-Lakeland_terrier', '../archive/images/train/n02090721-Irish_wolfhound', '../archive/images/train/n02096177-cairn', '../archive/images/train/n02106030-collie', '../archive/images/train/n02105251-briard', '../archive/images/train/n02102973-Irish_water_spaniel', '../archive/images/train/n02099601-golden_retriever', '../archive/images/train/n02088466-bloodhound', '../archive/images/train/n02094433-Yorkshire_terrier', '../archive/images/train/n02092339-Weimaraner', '../archive/images/train/n02110806-basenji', '../archive/images/train/n020

In [ ]:
for directory_path in glob.glob(train_path+"*"):
    label = directory_path.split("-")[-1]
    print(label)
    for img_path in glob.glob(os.path.join(directory_path, "*.jpg")):
        print(img_path)
        img = cv2.imread(img_path, cv2.IMREAD_COLOR)       
        img = cv2.resize(img, (SIZE, SIZE))
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        train_images.append(img)
        train_labels.append(label)

In [10]:
#Convert lists to arrays        
train_images = np.array(train_images)
train_labels = np.array(train_labels)


In [11]:
# Capture test/validation data and labels into respective lists

test_images = []
test_labels = [] 

In [12]:
for directory_path in glob.glob(validation_path+"*"):
    fruit_label = directory_path.split("-")[-1]
    for img_path in glob.glob(os.path.join(directory_path, "*.jpg")):
        img = cv2.imread(img_path, cv2.IMREAD_COLOR)
        img = cv2.resize(img, (SIZE, SIZE))
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        test_images.append(img)
        test_labels.append(fruit_label)


In [13]:
#Convert lists to arrays                
test_images = np.array(test_images)
test_labels = np.array(test_labels)

In [14]:
#Encode labels from text to integers.
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(test_labels)
test_labels_encoded = le.transform(test_labels)
le.fit(train_labels)
train_labels_encoded = le.transform(train_labels)

In [19]:
#Split data into test and train datasets (already split but assigning to meaningful convention)
x_train, y_train, x_test, y_test = train_images, train_labels_encoded, test_images, test_labels_encoded

In [ ]:
# Normalize pixel values to between 0 and 1
x_train, x_test = x_train / 255.0, x_test / 255.0